# Extraction of the data from Google Trends

In [36]:
import pandas as pd
import numpy as np
#Connect to API
from pytrends.request import TrendReq
from datetime import date
import datetime

In [3]:
import pandas as pd
import numpy as np
#Connect to API
from pytrends.request import TrendReq
from datetime import date
import datetime

In [2]:
today = str(datetime.date.today())
today

'2020-06-23'

## Function to automate the data extraction

In [3]:
def get_daily_data(category):
    """
    Get the data from Google Trends.
    I used and transformed slightly the code found on
    https://github.com/GeneralMills/pytrends/blob/master/pytrends/dailydata.py
    
    (GitHub, 2019)
    """
    # Start pytrends for the UK
    pytrend = TrendReq(hl='GB-ENG', tz=360)
    # Declare a var to store the search term
    #### build the playload
    categ=category
    
    pytrend.build_payload(
    kw_list=[''],
    # selection of a category
    cat=categ, 
    # everything since 2004
    timeframe='all',
    # for UK
    geo='GB',
    #default is web searches
    gprop='')
    
    data = pytrend.interest_over_time()
    
    df=pd.DataFrame(data)
    #drop the "isPartial column"
    df.drop(labels=['isPartial'],axis='columns', inplace=True)
    # Reset the index in the current dataframe, Like when ungrouping a pivot table, the former index will be put in a new column
    df.reset_index(inplace=True)
    # rename the column with the GT index by the variable/category number
    df.rename(columns={'':categ}, inplace=True)
    # Tranform date in month and year format
    df["date"]=pd.to_datetime(df["date"]).dt.strftime('%Y-%m')
    
    """
    Export the file to a csv
    """
    # export the file to CSV format with the date and number of the category
    root='17062020'
    df.to_csv(root+'/'+'GT_'+today+'_'+str(categ)+'.csv', encoding='utf_8_sig')
    
    return print('Done!')

In [4]:
import time
# We will monitor the time it takes to run the notebook
start = time.time()

List of variables for the categories

## Categories to extract

We will use the formula to extract the data for each categories we are interested in.

I split categories in groups, in order to identify eventual problems from the extraction which can come from the absence of data for a category.

In [51]:
categ_list1=[203,882,361,467,1190,907]

In [52]:
for i in categ_list1:
    get_daily_data(i)

Done!
Done!
Done!
Done!
Done!
Done!


In [53]:
categ_list21=[404,1191,573,906,472]
for i in categ_list21:
    get_daily_data(i)

Done!
Done!
Done!
Done!
Done!


In [54]:
categ_list4=[65,466,270,465,179,951,406, 249,124]
for i in categ_list4:
    get_daily_data(i)

Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!


In [55]:
categ_list5=[364,1150,909,382,273,560,276,1173,74]
for i in categ_list5:
    get_daily_data(i)

Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!


In [56]:
categ_list6=[263,123,50,138,89,405,71,658]
for i in categ_list6:
    get_daily_data(i)

Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!


In [5]:
categ_list7=[908, 68, 1349, 1293, 1292, 256, 124]
for i in categ_list7:
    get_daily_data(i)

Done!
Done!
Done!
Done!
Done!
Done!
Done!


In [4]:
categ_list8=[378, 646,1269]
for i in categ_list8:
    get_daily_data(i)

Done!
Done!
Done!


In [6]:
# Finish Timer
notebook_duration = round((time.time() - start)/60, 5)
print(f'The completion of scrapping GT data took {notebook_duration} minutes.')

The completion of scrapping GT data took 1.24422 minutes.


# Consolidation

In [4]:
#folder name
root='17062020'

In [5]:
import glob
import functools
#folder name
root = '17062020'
#The glob module finds all the pathnames matching a specified pattern according 
#to the rules used by the Unix shell, although results are returned in arbitrary order
all_files = glob.glob(root + "/*.csv")

li = []

for filename in all_files:
    dfs= [pd.read_csv(filename, index_col='date') for filename in all_files]
    df_final = functools.reduce(lambda left,right: pd.merge(left,right,on='date',how='inner'), dfs)

Final_df = df_final.loc[:,['50','65','68','71','74', '89','123','124','138','179','203','249','263',
                           '270', '273','276','361','364','378','382','404','405','406','465','466',
                           '467','472','560','573','646','658','882','906', '907','908','909','951',
                           '1150','1190','1191','1269','1292','1293','1349']]

Final_df.to_csv('GT_raw_data.csv', encoding='utf_8_sig')
Final_df

,50,65,68,71,74,89,123,124,138,179,...,908,909,951,1150,1190,1191,1269,1292,1293,1349
date,,,,,,,,,,,,,,,,,,,,,
2004-01,79,89,62,44,100,88,100,71,76,95,...,38,32,88,62,100,59,75,60,95,35
2004-02,78,86,60,45,94,86,87,65,72,89,...,40,28,76,61,85,57,76,55,81,37
2004-03,78,89,61,45,96,92,92,62,76,89,...,37,26,75,59,73,64,75,53,77,41
2004-04,76,88,61,42,86,92,85,61,68,86,...,34,26,73,60,82,77,80,46,80,33
2004-05,75,90,63,42,93,92,85,61,68,84,...,39,27,72,59,70,79,80,52,76,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02,43,59,74,73,41,58,62,68,86,38,...,62,64,77,59,34,32,40,71,57,20
2020-03,39,58,59,79,48,51,61,53,76,28,...,68,67,70,69,34,35,41,73,67,19
2020-04,38,61,66,100,36,49,63,52,76,18,...,100,100,83,96,26,72,39,93,61,18


In [6]:
Final_df.tail()

,50,65,68,71,74,89,123,124,138,179,...,908,909,951,1150,1190,1191,1269,1292,1293,1349
date,,,,,,,,,,,,,,,,,,,,,
2020-02,43,59,74,73,41,58,62,68,86,38,...,62,64,77,59,34,32,40,71,57,20
2020-03,39,58,59,79,48,51,61,53,76,28,...,68,67,70,69,34,35,41,73,67,19
2020-04,38,61,66,100,36,49,63,52,76,18,...,100,100,83,96,26,72,39,93,61,18
2020-05,40,68,76,95,37,58,85,60,83,20,...,97,90,94,94,28,100,45,98,67,19
2020-06,43,69,78,87,41,64,77,67,94,23,...,90,79,100,100,31,93,45,100,71,21
